In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
from astropy.cosmology import LambdaCDM 
from astropy import cosmology
#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [2]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [3]:
# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

# def match_tables_fib(t1,t2,match_tol):
#     if 'level_0' in t1.columns:
#         t1.drop(labels=['level_0'], axis=1, inplace=True)
#     t1.reset_index(drop=False, inplace=True)
#     if 'level_0' in t2.columns:
#         t2.drop(labels=['level_0'], axis=1, inplace=True)
#     t2.reset_index(inplace=True, drop=False)
#     t1['Table_flag'] = 'Table1'
#     t2['Table_flag'] = 'Table2'
#     # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
#     c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
#     c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
#     # Adding a match tolerance here, with user input for the function
#     max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
#     #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
#     idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
#     # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
#     # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
#     # a single set of coordinates at this point
#     # We need to make tables for t1 and t2 that do not include the matched items
#     t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
#     t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
#     # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
#     # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
#     # this with the primary table
#     tmatches = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name').reset_index(drop=True)
#     tunique = pd.concat([t1unique, t2unique]).sort_values(by='Name').reset_index(drop=True)
#     #
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'Paper(s)'] += " ; " + t2['Paper(s)'][0]
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'BibCode(s)'] += " ; " + t2['BibCode(s)'][0]
#     #t1matches.loc[t1matches['index'].isin(c1_dups['idx1']), 'DOI(s)'] += " ; " + t2['DOI(s)'][0]
#     return tunique, tmatches, idx1, idx2

# Here I'm rewriting our matching algorithm using the search_around_sky() function
# It may not always be the best option, but at least for these double peaked catalogs, I think I'm going to run 
# with it

def match_tables_fib(t1,t2,match_tol):
    if 'level_0' in t1.columns:
        t1.drop(labels=['level_0'], axis=1, inplace=True)
    t1.reset_index(drop=False, inplace=True)
    if 'level_0' in t2.columns:
        t2.drop(labels=['level_0'], axis=1, inplace=True)
    t2.reset_index(inplace=True, drop=False)
    t1['Table_flag'] = 'Table1'
    t2['Table_flag'] = 'Table2'
    # First we begin by matching RA1 and Dec1 of t1 to RA1 and Dec1 of t2
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA1_deg']*u.degree, dec=t2['Dec1_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1, idx2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_1 = pd.concat([(t1.iloc[idx1]),(t2.iloc[idx2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_1 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    # Now we need to match RA1 and Dec1 of t1 to RA2 and Dec2 of t2 in case the pairs are mixeup between tables
    c1 = SkyCoord(ra=t1['RA1_deg']*u.degree, dec=t1['Dec1_deg']*u.degree) # Storing coordinates for table 1
    c2 = SkyCoord(ra=t2['RA2_deg']*u.degree, dec=t2['Dec2_deg']*u.degree) # storing coordinates for table 2
    # Adding a match tolerance here, with user input for the function
    max_sep = match_tol * u.arcsec # The max match tolerance will be 5''
    #idx2, d2d2, d3d2 = match_coordinates_sky(c1, c2) # Now matching table 1 to table 2
    idx1_2, idx2_2, _, _ = c2.search_around_sky(c1, max_sep) 
    # idx1 and idx2 are the indices in table 1 and table 2 which are the closest matching rows to each other
    # Note, we should not need to cross match RA1 vs. RA2, across table because the double peaked sources only have
    # a single set of coordinates at this point
    # We need to make tables for t1 and t2 that do not include the matched items
    t1unique = (t1[~t1['index'].isin(idx1_2)]).reset_index(drop=True)
    t2unique = (t2[~t2['index'].isin(idx2_2)]).reset_index(drop=True)
    # And then we need a table for the matches items where we ensure they are properly matching (SDSS names should \
    # be the same), and then remove the duplicates, store the relevant info from the second table, and concatenate \
    # this with the primary table
    tmatches_2 = pd.concat([(t1.iloc[idx1_2]),(t2.iloc[idx2_2])]).sort_values(by='Name1').reset_index(drop=True)
    tunique_2 = pd.concat([t1unique, t2unique]).sort_values(by='Name1').reset_index(drop=True)
    #
    return tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2






In [4]:
#xliu2011 = pd.read_csv('Tables/xliu2011_catalog.csv', sep = ',')

xliu2011 = ((Table.read('Tables/XLiu2011b/table1.dat', readme = 'Tables/XLiu2011b/ReadMe', format='ascii.cds')).to_pandas())
print(len(xliu2011))

name_to_coords(xliu2011,xliu2011['SDSS'])

#xliu2011


2618


In [5]:
# Dropping the quintuple system from the pairs catalog now...
xliu2011 = xliu2011[xliu2011['SDSS']!='J095859.91+130308.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095900.42+130241.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095903.29+130221.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095906.68+130135.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095908.96+130352.5']

#Quadruples here
xliu2011 = xliu2011[xliu2011['SDSS']!='J135632.71+232118.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J135635.73+232136.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J135638.42+232151.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J135639.45+232210.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J144702.08+134006.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J144705.78+133953.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J144708.49+133917.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J144712.81+133939.3']

#Dropping all triples
xliu2011 = xliu2011[xliu2011['SDSS']!='J075925.27+375216.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J075931.83+375252.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J075932.30+375319.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J082846.95+122724.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J082847.93+122832.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J082851.50+122726.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J085927.64+173344.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J085930.05+173409.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J085931.39+173253.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J090515.73+075618.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J090516.12+075522.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J090520.84+075440.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095942.61+261914.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095947.21+261711.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J095948.15+261845.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J100722.96+230234.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J100725.21+230114.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J100732.36+230131.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102207.11+175017.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.61+175134.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102212.87+175113.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102223.90+154443.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.54+154514.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102228.76+154339.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102411.98+025234.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.08+025327.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102415.56+025241.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102546.26+134300.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102547.75+134456.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J102548.60+134340.6']


xliu2011 = xliu2011[xliu2011['SDSS']!='J105609.41+465341.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J105611.76+465250.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J105618.05+465155.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110441.92+041750.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110445.47+041755.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110449.56+041636.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110517.99+005328.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110520.67+005317.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110521.51+005256.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.72+025820.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110719.87+025926.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J110723.69+025829.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.18+032833.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J111005.60+032836.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J111007.99+032639.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J114938.17-033408.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J114939.73-033146.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J114949.79-033103.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120550.07+202837.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120551.47+203119.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120554.22+203421.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120718.36+204450.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120721.00+204503.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J120722.37+204353.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121425.18+082251.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121428.15+082225.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121431.24+082225.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.50+125924.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121442.95+125746.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J121443.11+125837.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J131628.81+525737.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J131629.82+525601.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J131637.18+525447.6']

xliu2011 = xliu2011[xliu2011['SDSS']!='J132356.94+305917.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J132359.96+305556.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J132404.73+305723.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J133227.93+071859.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J133232.26+071739.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J133242.55+071938.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141057.24+252950.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141102.54+253109.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141105.14+252857.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141151.61-000303.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.47-000302.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141153.81-000203.0']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141421.15+025721.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.72+025834.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J141422.99+025840.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143430.61+230529.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143438.51+230443.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143442.15+230637.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143801.02+101051.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143802.18+100944.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J143810.34+100805.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J150631.41+404146.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J150640.58+404134.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J150645.48+404051.5']

xliu2011 = xliu2011[xliu2011['SDSS']!='J150821.57+191131.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J150823.96+191229.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J150831.30+191103.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151310.42+042855.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.70+042701.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151311.80+042952.5']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151640.00+215938.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151641.90+215955.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J151644.79+215932.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160633.13+174632.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160638.60+174659.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160639.35+174540.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160709.88+183828.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160710.49+184004.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J160715.79+184140.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J162906.60+114231.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J162907.29+114331.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J162908.94+114447.8']
xliu2011 = xliu2011[xliu2011['SDSS']!='J163300.85+114510.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J163304.69+114431.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J163305.33+114532.4']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231138.88+150741.7']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231142.06+150638.3']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231143.99+150603.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231224.52+135704.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231225.99+135450.2']
xliu2011 = xliu2011[xliu2011['SDSS']!='J231229.22+135632.1']
xliu2011 = xliu2011[xliu2011['SDSS']!='J232402.98-095341.9']
xliu2011 = xliu2011[xliu2011['SDSS']!='J232403.39-095334.6']
xliu2011 = xliu2011[xliu2011['SDSS']!='J232407.99-095334.3']

# Correcting the sep, kpc, dV columns for two pairs here now so they'll be properly grouped...

#J233743.32-003106.5
#J233747.24-003056.9

#J024035.65-083249.6 first...
xliu2011.loc[xliu2011.SDSS=='J024035.65-083249.6', 'sep'] = 60.5
xliu2011.loc[xliu2011.SDSS=='J024035.65-083249.6', 'rp'] = 45.0
xliu2011.loc[xliu2011.SDSS=='J024035.65-083249.6', 'dV'] = 337

# And now J233743.32-003106.5 ....
xliu2011.loc[xliu2011.SDSS=='J233743.32-003106.5', 'sep'] = 59.6
xliu2011.loc[xliu2011.SDSS=='J233743.32-003106.5', 'rp'] = 42.4
xliu2011.loc[xliu2011.SDSS=='J233743.32-003106.5', 'dV'] = 164



In [6]:
print(len(xliu2011))

2488


In [7]:
#grouped.ngroup()
#len(grouped)
#grouped.first()
#print(grouped.groups)


In [8]:
#xliu2011['SDSS'].to_list()

In [9]:
xliu2011['GroupID'] = xliu2011.groupby(by=['sep','rp','dV']).ngroup()
xliu2011['GroupNum'] = xliu2011.groupby(by=['GroupID'])['GroupID'].transform('count')

# Adding a component ID column here, and then reassigning the component ID for the second row of each grouped pair
# So that we can filter the table into two dataframes, and then recombine in the format we want
xliu2011['ComponentID'] = 1
xliu2011.loc[xliu2011.groupby(by=['sep','rp','dV']).tail(1).index, 'ComponentID'] = 2

#Example code from stack exchange
#df.loc[df.groupby('colA').tail(1).index, 'colC'] = 'Current'

# Creating tables now:
tab1 = xliu2011[xliu2011['ComponentID']==1]
tab2 = xliu2011[xliu2011['ComponentID']==2]

#tab1 = tab1.columns=['SDSS1','Plate1','Fiber1','MJD1','z1','sep','rp','dV','rmag1','FAGN1','FTid1','RA1','Dec1','GroupID','GroupNum','ComponentID1']
#tab2 = tab2.columns['SDSS2','Plate2','Fiber2','MJD2','z2','sep','rp','dV','rmag2','FAGN2','FTid2','RA2','Dec2','GroupID','GroupNum','ComponentID2']
# 'SDSS','Plate','Fiber','MJD','z','sep','rp','dV','rmag','FAGN','FTid','RA','Dec','GroupID','GroupNum','ComponentID'

xliu2011 = tab1.join(tab2.set_index('GroupID'), on='GroupID', how='left', lsuffix = '1', rsuffix='2')



In [10]:
xliu2011.reset_index(drop=True, inplace=True)
print(len(xliu2011))
#xliu2011
#tab1
#tab2

1244


In [11]:
#xliu2011

In [12]:
#  HERE WE NEED TO COME BACK AND MAKE A MULTIS TABLE BASED ON TEH OBJEXTS WE REMOVED FROM LIU'S TABLE
# THIS WILL DEALT WITH SEPARATELY FROM THE DUALS TABLE

#multis = xliu2011[xliu2011['GroupNum']!=2].reset_index(drop=True)
#multis












In [13]:
xliu2011['System Type'] = 'Dual AGN Candidate'
xliu2011['Selection Method'] = 'Fiber Spectroscopy / Optical Spectroscopy / Optical Spectroscopic Emission Line Ratios'
xliu2011['Confirmation Method'] = ''

xliu2011['Name1'] = xliu2011['SDSS1']
xliu2011['Name2'] = xliu2011['SDSS2']

# Converting the coordinates
coordconvert = SkyCoord(ra = xliu2011['RA1'], dec = xliu2011['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
xliu2011['RA1_deg'] = coordconvert.ra.degree
xliu2011['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
coordconvert = SkyCoord(ra = xliu2011['RA2'], dec = xliu2011['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
xliu2011['RA2_deg'] = coordconvert.ra.degree
xliu2011['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
xliu2011['Equinox1'] = "J2000"
xliu2011['Coordinate_waveband1'] = "Optical"
xliu2011['Coordinate_Source1'] = "SDSS"
xliu2011['Equinox2'] = "J2000"
xliu2011['Coordinate_waveband2'] = "Optical"
xliu2011['Coordinate_Source2'] = "SDSS"

xliu2011.rename(columns={'dV1': 'dV'}, inplace=True)

# Recomputing separations here *after* changing the catalogue to have pairs each within a single row...
xliu2011['Sep'] = (SkyCoord(Angle(xliu2011['RA1']+' hours').degree,Angle(xliu2011['Dec1']+' degrees').degree, \
                    unit='deg', frame='icrs').separation(SkyCoord(Angle(xliu2011['RA2']+' hours').degree, \
                    Angle(xliu2011['Dec2']+' degrees').degree, unit='deg', frame='icrs'))).arcsecond

# Calculating now rp and dV for each row, and the redshift type for each row...
tab = Table.from_pandas(xliu2011)
tab['Sep(kpc)'] = tab['Sep']/(cosmo.arcsec_per_kpc_proper(tab['z1']))
xliu2011 = Table.to_pandas(tab)

xliu2011['dV_rwp'] = (2.99e+5)*((1+xliu2011['z1'])**2 - (1+xliu2011['z2'])**2)/((1+xliu2011['z1'])**2+(1+xliu2011['z2'])**2)
xliu2011['z1_type'] = 'spec'
xliu2011['z2_type'] = 'spec'

# Dropping unncessary columns here....

xliu2011_forflagging = xliu2011.copy()
xliu2011_forflagging.drop(labels=['Plate1','Fiber1','MJD1','sep1','rp1','GroupID','GroupNum1',\
                      'ComponentID1','Plate2','Fiber2','MJD2','sep2','rp2','dV2','GroupNum2',\
                      'ComponentID2'], inplace=True, axis=1)

# Plate, Fiber, MJD, GroupID, GroupNum, and Group Components 1 and 2, as well as sep2, dV2, and rp2.
# Once I establish how close my Sep calculation is to Liu's, I'll remove her column for that and rp/dV as well. 
xliu2011.drop(labels=['Plate1','Fiber1','MJD1','sep1','rp1','FAGN1','FTid1','GroupID','GroupNum1',\
                      'ComponentID1','Plate2','Fiber2','MJD2','sep2','rp2','dV2','FAGN2','FTid2','GroupNum2',\
                      'ComponentID2'], inplace=True, axis=1)

# Now adding in columns for DOI, Paper, ADS link...
xliu2011['Paper(s)'] = "Liu+2011b"
xliu2011['BibCode(s)'] = "2011ApJ...737..101L"
xliu2011['DOI(s)'] = "https://doi.org/10.1088/0004-637X/737/2/101"
xliu2011['Notes'] = ' '

#xliu2011

In [14]:
#xliu2011

In [15]:
# Secondary method for calculating the separations...

#red = xliu2011[['z1']].to_numpy()
#rat = cosmo.arcsec_per_kpc_proper(xliu2011[['z1']].to_numpy())**(-1)
#ratdf = pd.DataFrame(rat)
#
#xliu2011['rat'] = ratdf[0]
#xliu2011['Sep(kpc)'] = xliu2011['Sep']*xliu2011['rat']


In [16]:
xliu2011.to_csv('xliu2011_formated_catalog.csv', sep=',', index=False)
xliu2011_forflagging.to_csv('xliu2011_formated_catalog_forrflags.csv', sep=',', index=False)


In [17]:
#39+4+3

In [18]:
# The quintuiple from Liu+2011: J0959+1259 (as published by De Rosa+2015)

#****J095859.91+130308.4,1744,331,53055,0.0358,69.5,49.2,48,14.7,5,0,09:58:59.91,+13:03:08.4
#****J095900.42+130241.6,2584,81,54153,0.0351,47.6,33.7,153,14.7,5,0,09:59:00.42,+13:02:41.6
#****J095903.29+130221.0,1744,337,53055,0.0356,47.6,33.7,153,15.16,1,0,09:59:03.29,+13:02:21.0
#****J095906.68+130135.4,1744,332,53055,0.0366,68.3,48.4,290,15.38,0,0,09:59:06.68,+13:01:35.4
#****J095908.96+130352.5,2584,84,54153,0.0339,124.9,88.5,492,14.94,0,0,09:59:08.96,+13:03:52.5

# I can pull these out of the original list manually.


# The quadruples now...

#There's more spectra in this region now
#****J135632.71+232118.3
#****J135635.73+232136.0
#****J135638.42+232151.5
#****J135639.45+232210.3

#****J144702.08+134006.2
#****J144705.78+133953.1
#****J144708.49+133917.0
#****J144712.81+133939.3


# Triples now

# But this one is a quadruple spectra system. Not sure if it's a quadriple AGN tho
#****J075925.27+375216.0
#****J075931.83+375252.2
#****J075932.30+375319.5

# Again, more spectra available for this system
#****J082846.95+122724.1
#****J082847.93+122832.0
#****J082851.50+122726.8

#****J085927.64+173344.9
#****J085930.05+173409.2
#****J085931.39+173253.6

#This one has an AGN nearby that isn't cataloged by Liu
#****J090515.73+075618.9
#****J090516.12+075522.3
#****J090520.84+075440.1

#****J095942.61+261914.2
#****J095947.21+261711.4
#****J095948.15+261845.8

#****J100722.96+230234.1
#****J100725.21+230114.8
#****J100732.36+230131.7

#****J102207.11+175017.2
#****J102212.61+175134.8
#****J102212.87+175113.6

#****J102223.90+154443.1
#****J102228.54+154514.0
#****J102228.76+154339.0

#More soectra available now
#****J102411.98+025234.2
#****J102415.08+025327.5
#****J102415.56+025241.6

#More spectra available now
#****J102546.26+134300.7
#****J102547.75+134456.9
#****J102548.60+134340.6


#######################################################################

#J105609.41+465341.7
#J105611.76+465250.5
#J105618.05+465155.4
#
#J110441.92+041750.4
#J110445.47+041755.2
#J110449.56+041636.0
#
#J110517.99+005328.3
#J110520.67+005317.1
#J110521.51+005256.5
#
#J110719.72+025820.5
#J110719.87+025926.0
#J110723.69+025829.3
#
#J111005.18+032833.7
#J111005.60+032836.2
#J111007.99+032639.5
#
#J114938.17-033408.7
#J114939.73-033146.9
#J114949.79-033103.7

#This one seems to be a system of 5 or even upwards of 8. Need to revisit.
#J120537.43+203320.6
#J120540.09+203320.6
#J120550.07+202837.2
#J120551.47+203119.1
#J120554.22+203421.1

##But the triple seems to be 
#J120550.07+202837.2
#J120551.47+203119.1
#J120554.22+203421.1
#
#J120718.36+204450.9
#J120721.00+204503.5
#J120722.37+204353.6
#
#J121425.18+082251.9
#J121428.15+082225.6
#J121431.24+082225.7
#
#J121442.50+125924.8
#J121442.95+125746.4
#J121443.11+125837.4
#
#J131628.81+525737.2
#J131629.82+525601.2
#J131637.18+525447.6

#######################################################################

#J132356.94+305917.0
#J132359.96+305556.7
#J132404.73+305723.5
#
#J133227.93+071859.2
#J133232.26+071739.1
#J133242.55+071938.1
#
#J141057.24+252950.0
#J141102.54+253109.7
#J141105.14+252857.9
#
#J141151.61-000303.2
#J141153.47-000302.3
#J141153.81-000203.0
#
#J141421.15+025721.2
#J141422.72+025834.5
#J141422.99+025840.8
#
#J143430.61+230529.9
#J143438.51+230443.2
#J143442.15+230637.5
#
#J143801.02+101051.5
#J143802.18+100944.5
#J143810.34+100805.8
#
#J150631.41+404146.4
#J150640.58+404134.9
#J150645.48+404051.5

#######################################################################


#J150821.57+191131.6
#J150823.96+191229.2
#J150831.30+191103.6
#
## A whole lot of objects/pairs in this area, but i think the triple is actually:
#J151310.42+042855.5
#J151311.70+042701.6
#J151311.80+042952.5
#
#J151640.00+215938.7
#J151641.90+215955.7
#J151644.79+215932.2
#
#J160633.13+174632.3
#J160638.60+174659.8
#J160639.35+174540.3
#
#J160709.88+183828.6
#J160710.49+184004.7
#J160715.79+184140.8
#
#J162906.60+114231.2
#J162907.29+114331.2
#J162908.94+114447.8
#
#J163300.85+114510.1
#J163304.69+114431.3
#J163305.33+114532.4
#
#J231138.88+150741.7
#J231142.06+150638.3
#J231143.99+150603.1
#
#J231224.52+135704.6
#J231225.99+135450.2
#J231229.22+135632.1
#
#J232402.98-095341.9
#J232403.39-095334.6
#J232407.99-095334.3



In [19]:
# These two appear to make a pair, but they have different sep, kpc, and dV listed, despite the separation listed for the
# second object making sense but the first object doesnt make sense. I have a feeling this is a typo.
#J233743.32-003106.5
#J233747.24-003056.9

# These two also appear to make a bound pair:
#J024035.65-083249.6
#J024037.96-083200.1

#The strange thing now is that Liu's tabulation does not add up. She does not appear to have 39 triples... it seems
# like 38 instead...


#So what I'll need to do for these objects is go back in and replace the values for sep, kpc, dV, and also the GroupID
# and GroupNum columns


In [20]:
# Adding in the information from Barrows+

barrowsobjs = ['J080523.29+281815.8','J090714.45+520343.4','J105842.58+314459.8','J111519.98+542316.7']
#J080523.29+281815.84 J090714.45+520343.42 J105842.58+314459.76 --> this is the secondary! J111519.98+542316.65 --> this is this the secondary


## Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in liu2011.iterrows():
#    if row['SDSS1'] in drobjs:
#        xliu2011.at[index, 'Paper(s)'] += ' ; Barrows+2016'
#        xliu2011.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
#        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
#        #xliu2011.at[index, 'System Type']='Dual AGN'
#        xliu2011.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
#        #xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'
#    
#for index, row in liu2011.iterrows():
#    if row['SDSS2'] in drobjs:
#        xliu2011.at[index, 'Paper(s)'] += ' ; Barrows+2016'
#        xliu2011.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
#        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
#        #xliu2011.at[index, 'System Type']='Dual AGN'
#        xliu2011.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
#        #xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'
#print(len(barrowsobjs))
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in barrowsobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; Barrows+2016'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
        #xliu2011.at[index, 'System Type']='Dual AGN'
        xliu2011.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
        #xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'
    if row['SDSS2'] in barrowsobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; Barrows+2016'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2016ApJ...829...37B' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/0004-637X/829/1/37'
        #xliu2011.at[index, 'System Type']='Dual AGN'
        xliu2011.at[index, 'Notes']+=' Selected by Barrows+ as an offset AGN candidate.'
        #xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

for i in xliu2011['Paper(s)']:
    if 'Barrows+2016' in i:
        print('True')

# verified


True
True
True
True


In [21]:
#xliu2011

In [22]:
# Here we are loading in information from the De Rosa paper

drobjs = ['J094554.41+423840.0','J094554.49+423818.7','J103853.29+392151.2','J103855.95+392157.6','J162640.93+142243.6','J162644.51+142250.7','J145627.40+211956.0','J145631.36+212030.1']
# 'J103853.28+392151.1' --> designations used by De Rosa+2018; I've manually adjusted these
# 'J103855.94+392157.5' --> designations used by De Rosa+2018; I've manually adjusted these

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; De Rosa+2018'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2018MNRAS.480.1639D' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty1867'
        xliu2011.at[index, 'System Type']='Dual AGN'
        xliu2011.at[index, 'Notes']='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

for index, row in xliu2011.iterrows():
    if row['SDSS2'] in drobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; De Rosa+2018'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2018MNRAS.480.1639D' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty1867'
        xliu2011.at[index, 'System Type']='Dual AGN'
        xliu2011.at[index, 'Notes']='AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'

        
drobjs = ['J094554.41+423840.0','J094554.49+423818.7']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='Unobscured and Compton-think pair. Seyfert+Seyfert pair.'

drobjs = ['J103853.29+392151.2','J103855.95+392157.6']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='Unobscured and Compthin-thick pair. Seyfert+Seyfert pair.'

drobjs = ['J162640.93+142243.6','J162644.51+142250.7']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='Compton-thick candidate and Compton-thin pair. Seyfert+LINER pair.'

drobjs = ['J145627.40+211956.0','J145631.36+212030.1']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='Compton-thin and Compton-thick pair. Seyfert+Seyfert pair.'

# verified that this matching process works


In [23]:
# Loading in the catalog from Liu+2010a now...
fu2018 = pd.read_csv('Tables/Fu2018/Fu2018.csv')#.drop(columns=['---'])
# Note here that any cells containing '--' in Xin's table are going to be replaced by 0's

# Fu+ overlaps with Liu+2011 and Ellison+2017, Make sure to mark these!
fu2018 = fu2018[fu2018['Plate'] != 8329]
#8083–9101, 8133–12704, and 8612–12705 --> in Liu+2011 and Fu+ consider as dual candidates
#8549–12705 and 9049–12701 --> in Liu+2011 but Fu+ do not consider  as dual candidates

# ***********************************
# go back and flag fu+2018 objects in Liu table using SDSS designations!!!
# ***********************************


coords = SkyCoord(ra=fu2018['R.A.1']*u.degree, dec=fu2018['Decl.1']*u.degree, frame='icrs')
# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)
# Concatenate to form J_format using a loop
fu2018['Name1'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]

coords = SkyCoord(ra=fu2018['R.A.2']*u.degree, dec=fu2018['Decl.2']*u.degree, frame='icrs')
# Convert RA and Dec to the desired format
ra_format = coords.ra.to_string(unit=u.hour, sep='', precision=2, pad=True)
dec_format = coords.dec.to_string(unit=u.deg, sep='', precision=2, alwayssign=True, pad=True)
# Concatenate to form J_format using a loop
fu2018['Name2'] = ["J" + ra + dec for ra, dec in zip(ra_format, dec_format)]

fu2018['z1'] = fu2018['Redshift1']
fu2018['z2'] = fu2018['Redshift2']
fu2018['z1_type'] = "spec"
fu2018['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(fu2018,fu2018['Name'])

# Converting the coordinates
# Convert Ra and Dec from degrees to sexagesimal format
coords = SkyCoord(ra=fu2018['R.A.1']*u.degree, dec=fu2018['Decl.1']*u.degree, frame='icrs')
fu2018['RA1'] = coords.ra.to_string(u.hour, sep=':', precision=2)
fu2018['Dec1'] = coords.dec.to_string(u.deg, sep=':', precision=2)

fu2018['RA1_deg'] = fu2018['R.A.1']
fu2018['Dec1_deg'] = fu2018['Decl.1']

coords = SkyCoord(ra=fu2018['R.A.2']*u.degree, dec=fu2018['Decl.2']*u.degree, frame='icrs')
fu2018['RA2'] = coords.ra.to_string(u.hour, sep=':', precision=2)
fu2018['Dec2'] = coords.dec.to_string(u.deg, sep=':', precision=2)

fu2018['RA2_deg'] = fu2018['R.A.2']
fu2018['Dec2_deg'] = fu2018['Decl.2']

# Adding details about the coordinates
fu2018['Equinox1'] = "J2000"
fu2018['Coordinate_waveband1'] = "Optical"
fu2018['Coordinate_Source1'] = "SDSS"
fu2018['Equinox2'] = "J2000"
fu2018['Coordinate_waveband2'] = "Optical"
fu2018['Coordinate_Source2'] = "SDSS"

fu2018['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
fu2018['Brightness1'] = -100
fu2018['Brightness_band1'] = "-100"
fu2018['Brightness_type1'] = "-100"

fu2018['Brightness2'] = -100
fu2018['Brightness_band2'] = "-100"
fu2018['Brightness_type2'] = "-100"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
fu2018['Sep'] = fu2018['Sep_as'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit

#fu2018['Sep(kpc)'] = fu2018['Sep']*((cosmo.arcsec_per_kpc_proper(fu2018['z']))**(-1))

# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#fu2018['delta_z'] = fu2018['z']-fu2018['z2']
fu2018['dV'] = (2.99e+5)*((1+fu2018['z1'])**2 - (1+fu2018['z2'])**2)/((1+fu2018['z1'])**2+(1+fu2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
fu2018['Selection Method'] = "Optical Spectroscopy / IFU Optical Spectroscopy" #DPSELs
fu2018['Confirmation Method'] = "-99"
fu2018['Paper(s)'] = "Fu+2018"
fu2018['BibCode(s)'] = "2018ApJ...856...93F"
fu2018['DOI(s)'] = "https://doi.org/ 10.3847/1538-4357/aab364"

fu2018['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#fu2018.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#fu2018 = fu2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#fu2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

fu2018

,Plate,IFU,Sample,Sep_as,rp,delta_v,R.A.1,Decl.1,Redshift1,stellar_disp1,log(M)1,log(L[OIII])1,Class1,R.A.2,Decl.2,Redshift2,stellar_disp2,log(M)2,log(L[OIII])2,Class2,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,7443,12703,1,6.9,5.5,91.9,229.52558,42.74585,0.04044,205.1,9.2,40.6,2,229.52653,42.74407,0.04074,126.0,9.3,39.9,2,J151806.14+424445.06,J151806.37+424438.65,0.04044,0.04074,spec,spec,15:18:06.14,42:44:45.06,229.52558,42.74585,15:18:06.37,42:44:38.65,229.52653,42.74407,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,6.9,-86.201096,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
1,7975,12704,2,3.5,5.7,178.6,324.58641,11.34867,0.08961,160.3,9.8,39.8,2,324.58655,11.34961,0.08902,154.7,9.1,39.3,2,J213820.74+112055.21,J213820.77+112058.60,0.08961,0.08902,spec,spec,21:38:20.74,11:20:55.21,324.58641,11.34867,21:38:20.77,11:20:58.60,324.58655,11.34961,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.5,161.945798,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
2,8083,9101,1,7.0,5.3,122.4,50.13841,-0.33996,0.03828,174.0,10.1,39.5,2,50.14021,-0.33923,0.03788,62.2,9.3,39.1,3,J032033.22-002023.86,J032033.65-002021.23,0.03828,0.03788,spec,spec,3:20:33.22,-0:20:23.86,50.13841,-0.33996,3:20:33.65,-0:20:21.23,50.14021,-0.33923,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,7.0,115.212696,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
3,8133,12704,2,3.7,8.8,36.9,114.77573,44.40277,0.13425,246.6,9.7,39.6,3,114.77430,44.40287,0.13437,177.6,9.3,39.4,3,J073906.18+442409.97,J073905.83+442410.33,0.13425,0.13437,spec,spec,7:39:06.18,44:24:09.97,114.77573,44.40277,7:39:05.83,44:24:10.33,114.77430,44.40287,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.7,-31.631564,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
4,8134,3701,3,3.7,8.6,547.6,113.32663,46.34819,0.12719,252.0,9.9,39.0,3,113.32576,46.34904,0.12902,233.0,9.5,39.0,2,J073318.39+462053.48,J073318.18+462056.54,0.12719,0.12902,spec,spec,7:33:18.39,46:20:53.48,113.32663,46.34819,7:33:18.18,46:20:56.54,113.32576,46.34904,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.7,-485.034319,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
5,8255,6101,2,3.6,4.1,7.5,166.50988,43.17348,0.05841,191.8,10.1,40.0,3,166.50906,43.17428,0.05843,197.1,9.4,39.0,3,J110602.37+431024.53,J110602.17+431027.41,0.05841,0.05843,spec,spec,11:06:02.37,43:10:24.53,166.50988,43.17348,11:06:02.17,43:10:27.41,166.50906,43.17428,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.6,-5.649931,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
6,8262,12705,2,13.3,18.9,127.5,185.49815,44.14566,0.07486,137.8,9.6,39.0,2,185.49350,44.14407,0.07529,95.2,9.1,39.0,4,J122159.56+440844.38,J122158.44+440838.65,0.07486,0.07529,spec,spec,12:21:59.56,44:08:44.38,185.49815,44.14566,12:21:58.44,44:08:38.65,185.49350,44.14407,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,13.3,-119.591652,Optical Spectroscopy / IFU Optical Spectroscopy,-99,Fu+2018,2018ApJ...856...93F,https://doi.org/ 10.3847/1538-4357/aab364,
8,8332,1

In [24]:
# Here we're going to match together the xliu2011 table and the Fu+2018 table and see if there are any matches

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(xliu2011,fu2018,3)

print(len(tmatches_1),len(tmatches_2))
# same objects picked up in tmatches_1 and 2, except tmatches_1 pickes up a third object

# it appears that AGN1 and 2 for J032033.21-002024.0 and J170024.36+382106.3 match across Liu and Fu
# AGNs 1 and 2 are flipped for J073905.83+442410.4
# taking this into account below...

# We're sticking with Liu's coordinates, separations
# Since redshifts were derived from IFU spectroscopy, we'll defer to Fu+ for redshifts and velocity differences/ \
# I'll recalculate dV's based on Fu's redshifts
# Also remember that Fu+ has stellar velocity dispersions if we want those
# We need to add in the selection method from Fu+

for i, j in zip(idx1, idx2):
    xliu2011.at[i, 'z1'] = fu2018.at[j, 'z1']
    xliu2011.at[i, 'z2'] = fu2018.at[j, 'z2']
    xliu2011.at[i, 'Selection Method'] += " / " + str(fu2018.at[j, 'Selection Method'])
    xliu2011.at[i, 'dV'] = fu2018.at[j, 'dV']
    xliu2011.at[i, 'Paper(s)'] += " ; " + fu2018.at[j, 'Paper(s)']
    xliu2011.at[i, 'BibCode(s)'] += " ; " + fu2018.at[j, 'BibCode(s)']
    xliu2011.at[i, 'DOI(s)'] += " ; " + fu2018.at[j, 'DOI(s)']
    xliu2011.at[i, 'Notes'] += fu2018.at[j, 'Notes']

for i, j in zip(idx1_2, idx2_2):
    xliu2011.at[i, 'z1'] = fu2018.at[j, 'z2'] # accounting for the AGNs being flipped between the two catalogs
    xliu2011.at[i, 'z2'] = fu2018.at[j, 'z1'] # accounting for the AGNs being flipped between the two catalogs
    xliu2011.at[i, 'Selection Method'] += " / " + str(fu2018.at[j, 'Selection Method'])
    xliu2011.at[i, 'dV'] = fu2018.at[j, 'dV']
    xliu2011.at[i, 'Paper(s)'] += " ; " + fu2018.at[j, 'Paper(s)']
    xliu2011.at[i, 'BibCode(s)'] += " ; " + fu2018.at[j, 'BibCode(s)']
    xliu2011.at[i, 'DOI(s)'] += " ; " + fu2018.at[j, 'DOI(s)']
    xliu2011.at[i, 'Notes'] += fu2018.at[j, 'Notes']

fu2018.drop(idx2)
fu2018.drop(idx2_2).reset_index(drop=True, inplace=True)

# verified that this process works

4 2


In [25]:
#tmatches_1

#for i in xliu2011['Paper(s)']:
#    if 'Fu+2018' in i:
#        print('True')

In [26]:
#tmatches_2

In [27]:
# concantenating Fu+ and Liu+ here

xliu2011 = pd.concat([xliu2011,fu2018]).reset_index(drop=True)


In [28]:
#xliu2011

In [29]:
# Here we're adding in the information from Hou+2019


drobjs = ['J090714.45+520343.4','J090714.61+520350.7','J080523.29+281815.8','J080523.40+281814.1','J133031.75-003611.9','J133032.00−003613.5','J105842.44+314457.6','J105842.58+314459.8','J154403.45+044607.5','J154403.67+044610.1']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; Hou+2019'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2019ApJ...882...41H' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/ab3225'
        #xliu2011.at[index, 'Notes'] = 'AGNs confirmed via X-rays and reanalyses of optical spectroscopy.'
        xliu2011.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy / Optical Spectroscopy'


drobjs = ['J090714.45+520343.4','J090714.61+520350.7']
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        xliu2011.at[index, 'System Type'] = 'Dual AGN'
        xliu2011.at[index, 'Notes']+='Type 2+Type 2 pair. Two Chandra X-ray point sources detected. One is Compton-thin. The other has too few counts for spectral analysis. X-ray luminosities are too high to explain via SF alone. Consistent with dual AGN scenario.'

drobjs = ['J080523.29+281815.8','J080523.40+281814.1']
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        xliu2011.at[index, 'Notes']+='Type 2+ Type2/LINER pair. NW nucleus detected in both Chandra bands and SE nucleus is not detected at all. NW is consistent with an AGN. SE galaxy optically consistent with post-starburst galaxy. Upper limits on X-ray luminosity do not rule out the presence of an AGN in SE galaxy. Possibility remains that NW galaxy ionizes both nuclei.'

drobjs = ['J133031.75-003611.9','J133032.00-003613.5']
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        xliu2011.at[index, 'Notes']+='HII/AGN composite + Type 2 pair. SE detected by Chandra and NW not detected. NW optically consistent with post-starburst. SE X-ray lum similar to expectation from SF but underlying AGN component cannot be ruled out. Upper limits on X-ray lums do not rule out presence of AGN in NW galaxy. Possibility remains that SE nucleus ionizes both nuclei.'

drobjs = ['J105842.44+314457.6','J105842.58+314459.8']
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        xliu2011.at[index, 'Notes']+='HII/AGN composite + Type 2 pair. NE detected by Chandra and is Compton-thin while SW not detected. X-ray source in NE consistent with AGN. X-ray upper limits do not rule out presence of AGN in SW. Possibility remains that NE ionizes both nuclei.'

drobjs = ['J154403.45+044607.5','J154403.67+044610.1']
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        xliu2011.at[index, 'System Type'] = 'Dual AGN'
        xliu2011.at[index, 'Notes']+='LINER+ LINER/Type 2/Composite pair. NE detected by Chandra in both hard and soft bands. SW detected only in soft band. NE is Compton-thin and consistent with an AGN. SW may be consistent with AGN as well because soft X-ray luminosity is well in excess of expectations from SF.'

# verified that this matching process works
# but note that 5 objects get matched in SDSS1 but only 3 or 4 objects get matched to SDSS2. 
# So a couple of names aren't quite right. Doesn't affect this too much, but we may want to consider matching by \
# RA and Dec to make any changes easier in the future


In [30]:
#xliu2011

In [31]:
# Here we're adding in information from Husemann+2020

drobjs = ['J031722.06+004801.8','J080523.29+281815.8','J085837.68+182223.4','J094741.58+633939.2',\
          'J111519.98+542316.7','J160933.41+283058.4','J164658.48+241134.1']

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
for index, row in xliu2011.iterrows():
    if row['SDSS1'] in drobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; Husemann+2020'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2020A&A...639A.117H' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/202037988'
        #xliu2011.at[index, 'System Type'] = 'Dual AGN'
        xliu2011.at[index, 'Notes']=' .'
        xliu2011.at[index, 'Confirmation Method'] = 'Optical Spectroscopy'

for index, row in xliu2011.iterrows():
    if row['SDSS2'] in drobjs:
        #print('True')
        xliu2011.at[index, 'Paper(s)'] += ' ; Husemann+2020'
        xliu2011.at[index, 'BibCode(s)'] += ' ; 2020A&A...639A.117H' 
        xliu2011.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1051/0004-6361/202037988'
        #xliu2011.at[index, 'System Type'] = 'Dual AGN'
        xliu2011.at[index, 'Notes']=' .'
        xliu2011.at[index, 'Confirmation Method'] = 'Optical Spectroscopy'

        
drobjs = ['J031722.06+004801.8']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. MODS spectra show that both are still remain Seyfert/AGN pairs but the EQW of the Halpha lines are low and in the weak AGN regime on the WHAN diagram.'

drobjs = ['J080523.29+281815.8']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. MODS spectra show that both are still remain Seyfert/AGN pairs but the EQW of the Halpha lines are low and in the weak AGN regime on the WHAN diagram. X-ray spctrum is rather soft.'

drobjs = ['J085837.68+182223.4']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. Peak of OIII emission is centered between the nuclei. MODS spectra reveal one nucleus is SF dominated. Bulk of the X-ray emission is detected between the nuclei. Spectrum is rather soft.'

drobjs = ['J094741.58+633939.2']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. MODS spectra show secondary nucleus falls into lower ionization regime and is most likely powered by stellar pop instead of AGN given low EQW of Halpha on WHAN diagram.'

drobjs = ['J111519.98+542316.7']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='MODS reveals AGN ionized gas along the entire slit indicating that the ionisation cone is pointing toward the companion. This suggests the primary is ionizing both nuclei.'

drobjs = ['J160933.41+283058.4']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. This system retains an optical dual AGN classification. But the BPT diagram does suggest a significant contribution from SF or LINER-like emission. This is inconsistent with the original Seyfert-Seyfert classification.'

drobjs = ['J164658.48+241134.1']
for index, row in xliu2011.iterrows():
    if (row['SDSS1'] in drobjs) or (row['SDSS2'] in drobjs):
        xliu2011.at[index, 'Notes']+='BPT ratios of secondary nucleus changes when viewed with MODS. Fiber spillover effects likely. MODS spectra show secondary nucleus falls into lower ionization regime and is most likely powered by stellar pop instead of AGN given low EQW of Halpha on WHAN diagram.'

# verified that this matching process works


In [32]:
#for i in xliu2011['Paper(s)']:
#    if 'Husemann+2020' in i:
#        print('True')

In [33]:
# Now we're going to load in the Fu+2015 tables here

In [34]:
def name_to_coords(df,dfcol):
    if (len(dfcol[0])) == 14:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=6) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=6, stop=13) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=6) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=8) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #print(dfcol.apply(len))
    elif (len(dfcol[0])) == 19:
        df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    elif (len(dfcol[0])) == 18:
        #print('True')
        df['Coordinates'] = dfcol#.str#.slice(start=1) # Stripping the J
        df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
        df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=18) # Stripping the RA parts
        df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
        df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
        df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
        return
    #elif dfcol.apply(len) ==
    else:
        print('Error Encountered')
        

In [35]:
# Loading in here the tables for Fu+2015, Fu+2015, and Gross+2019
# These are not necessarily related to Liu+2011 (although I think there is some overlap), nor is it directly \
# related to spectroscopic selection (these are radio selected), these papers do use optical spectra, so I'm \
# just loading them in here. We can shift it around later if we want. 

fu2015t1 = (Table.read('Tables/fu2015/table1.dat', readme = 'Tables/fu2015/ReadMe', format='ascii.cds')).to_pandas()
fu2015t2 = (Table.read('Tables/fu2015/table2.dat', readme = 'Tables/fu2015/ReadMe', format='ascii.cds')).to_pandas()
fu2015t3 = (Table.read('Tables/fu2015/table3.dat', readme = 'Tables/fu2015/ReadMe', format='ascii.cds')).to_pandas()

secondary_objs = fu2015t1.iloc[1::2]#.reset_index(drop=True)
# Get the indices of rows in every_other_df
secondary_objs_indices = secondary_objs.index
# Drop rows from original_df
fu2015t1 = fu2015t1.drop(secondary_objs_indices).reset_index(drop=True)
secondary_objs = secondary_objs.reset_index(drop=True)
# Renaming columns for the secondary objects
secondary_objs.rename(columns={'RName': 'RName2','S1.4pk':'S1.4pk2', 'e_S1.4pk':'e_S1.4pk2', 'S1.4int':'S1.4int2','OName':'OName2','Sep':'Sep2','umag':'umag2','gmag':'gmag2','rmag':'rmag2','imag':'imag2',\
                                        'zmag':'zmag2','z':'z2','Inst':'Inst2'}, inplace=True)
fu2015t1.rename(columns={'RName': 'RName1','S1.4pk':'S1.4pk1', 'e_S1.4pk':'e_S1.4pk1', 'S1.4int':'S1.4int1','OName':'OName1','Sep':'Sep','umag':'umag1','gmag':'gmag1','rmag':'rmag1','imag':'imag1',\
                                        'zmag':'zmag1','z':'z1','Inst':'Inst1'}, inplace=True)

fu2015t1 = pd.concat([fu2015t1,secondary_objs], axis=1).reset_index(drop=True)
fu2015t1['Notes'] = 'Grade A targets from Fu+2015.'

#df
#fu2015t1


In [36]:
#fu2015t2

In [37]:
secondary_objs = fu2015t2.iloc[1::2]#.reset_index(drop=True)
# Get the indices of rows in every_other_df
secondary_objs_indices = secondary_objs.index
# Drop rows from original_df
fu2015t2 = fu2015t2.drop(secondary_objs_indices).reset_index(drop=True)
secondary_objs = secondary_objs.reset_index(drop=True)
# Renaming columns for the secondary objects
secondary_objs.rename(columns={'RName': 'RName2','S1.4pk':'S1.4pk2', 'e_S1.4pk':'e_S1.4pk2', 'S1.4int':'S1.4int2','OName':'OName2','Sep':'Sep2','umag':'umag2','gmag':'gmag2','rmag':'rmag2','imag':'imag2',\
                                        'zmag':'zmag2','z':'z2','Inst':'Inst2'}, inplace=True)
fu2015t2.rename(columns={'RName': 'RName1','S1.4pk':'S1.4pk1', 'e_S1.4pk':'e_S1.4pk1', 'S1.4int':'S1.4int1','OName':'OName1','Sep':'Sep','umag':'umag1','gmag':'gmag1','rmag':'rmag1','imag':'imag1',\
                                        'zmag':'zmag1','z':'z1','Inst':'Inst1'}, inplace=True)

fu2015t2 = pd.concat([fu2015t2,secondary_objs], axis=1).reset_index(drop=True)
fu2015t2['Notes'] = 'Grade B targets from Fu+2015.'
#fu2015t2



In [38]:
#len(fu2015['Name2'][0])

In [39]:
fu2015 = pd.concat([fu2015t1,fu2015t2]).reset_index(drop=True)

fu2015['Name1'] = fu2015['RName1']
fu2015['Name2'] = fu2015['RName2']


#fu2015


In [40]:
fu2015 = pd.concat([fu2015t1,fu2015t2]).reset_index(drop=True)

fu2015['Name1'] = fu2015['RName1']
fu2015['Name2'] = fu2015['RName2']
#fu2015['z2'] = fu2015['z']
#fu2015['z1_type'] = "spec"
#fu2015['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
name_to_coords(fu2015,fu2015['Name1'])
fu2015.rename(columns={'RA': 'RA1','Dec':'Dec1'}, inplace=True)

name_to_coords(fu2015,fu2015['Name2'])
fu2015.rename(columns={'RA': 'RA2','Dec':'Dec2'}, inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2015['RA1'], dec = fu2015['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
fu2015['RA1_deg'] = coordconvert.ra.degree
fu2015['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#fu2015['RA2'] = fu2015['RA']
#fu2015['Dec2'] = fu2015['Dec']

coordconvert = SkyCoord(ra = fu2015['RA2'], dec = fu2015['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
fu2015['RA2_deg'] = coordconvert.ra.degree
fu2015['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
fu2015['Equinox1'] = "J2000"
fu2015['Coordinate_waveband1'] = "Optical"
fu2015['Coordinate_Source1'] = "SDSS"
fu2015['Equinox2'] = "J2000"
fu2015['Coordinate_waveband2'] = "Optical"
fu2015['Coordinate_Source2'] = "SDSS"

fu2015['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
fu2015['Brightness1'] = -100
fu2015['Brightness_band1'] = "-100"
fu2015['Brightness_type1'] = "-100"

fu2015['Brightness2'] = -100
fu2015['Brightness_band2'] = "-100"
fu2015['Brightness_type2'] = "-100"

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#fu2015['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#fu2015['Sep(kpc)'] = fu2015['Sep']*((cosmo.arcsec_per_kpc_proper(fu2015['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#fu2015['delta_z'] = fu2015['z']-fu2015['z2']
#fu2015['dV'] = (2.99e+5)*((1+fu2015['z'])**2 - (1+fu2015['z2'])**2)/((1+fu2015['z'])**2+(1+fu2015['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
fu2015['Selection Method'] = "Radio Imaging / Double Radio Sources" #DPSELs
fu2015['Confirmation Method'] = ""
fu2015['Paper(s)'] = "Fu+2015"
fu2015['BibCode(s)'] = "2015ApJ...799...72F"
fu2015['DOI(s)'] = "https://doi.org/10.1088/0004-637X/799/1/72"

#fu2015['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#fu2015.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#fu2015 = fu2015[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#fu2015.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

fu2015


,RName1,S1.4pk1,e_S1.4pk1,S1.4int1,OName1,Sep,umag1,gmag1,rmag1,imag1,zmag1,z1,Inst1,RName2,S1.4pk2,e_S1.4pk2,S1.4int2,OName2,Sep2,umag2,gmag2,rmag2,imag2,zmag2,z2,Inst2,Notes,Name1,Name2,RA1,Dec1,RA2,Dec2,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,004437.33+010132.5,0.38,0.07,0.96,004437.33+010132.5,4.6,NaN,24.1,23.4,24.9,NaN,NaN,NaN,004437.48+010136.4,2.02,0.07,3.99,004437.48+010136.5,4.6,NaN,NaN,NaN,23.0,22.2,NaN,NaN,Grade A targets from Fu+2015.,004437.33+010132.5,004437.48+010136.4,00:44:37.33,+01:01:32.5,00:44:37.48,+01:01:36.4,11.155542,1.025694,11.156167,1.026778,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
1,005043.06-003045.2,4.31,0.05,15.36,005043.07-003045.5,2.3,NaN,23.8,23.6,22.9,NaN,NaN,NaN,005043.18-003043.6,3.09,0.05,2.96,005043.16-003043.6,2.3,NaN,24.3,22.9,21.9,21.0,NaN,NaN,Grade A targets from Fu+2015.,005043.06-003045.2,005043.18-003043.6,00:50:43.06,-00:30:45.2,00:50:43.18,-00:30:43.6,12.679417,-0.512556,12.679917,-0.512111,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
2,005113.93+002047.0,0.34,0.05,0.61,005113.93+002047.2,3.4,20.0,18.6,17.1,15.3,16.8,0.1124,SDSS,005114.10+002049.5,2.50,0.05,3.47,005114.11+002049.4,3.4,20.0,17.8,17.1,15.2,16.4,0.1126,SDSS,Grade A targets from Fu+2015.,005113.93+002047.0,005114.10+002049.5,00:51:13.93,+00:20:47.0,00:51:14.10,+00:20:49.5,12.808042,0.346389,12.808750,0.347083,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
3,011156.46-000015.1,11.06,0.66,16.80,011156.44-000015.1,2.5,23.5,23.0,22.9,22.4,22.2,NaN,NaN,011156.58-000017.6,23.19,0.68,41.18,011156.52-000017.3,2.5,24.0,22.3,21.0,20.5,20.2,NaN,NaN,Grade A targets from Fu+2015.,011156.46-000015.1,011156.58-000017.6,01:11:56.46,-00:00:15.1,01:11:56.58,-00:00:17.6,17.985250,-0.004194,17.985750,-0.004889,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
4,012839.28+011309.4,1.41,0.09,2.13,012839.29+011309.3,4.3,NaN,25.2,23.6,22.7,21.5,NaN,NaN,012839.42+011312.4,0.90,0.09,2.17,012839.44+011313.0,4.3,NaN,25.7,24.8,24.3,NaN,NaN,NaN,Grade A targets from Fu+2015.,012839.28+011309.4,012839.42+011312.4,01:28:39.28,+01:13:09.4,01:28:39.42,+01:13:12.4,22.163667,1.219278,22.164250,1.220111,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
5,013412.78-010729.6,5.47,0.54,8.14,013412.78-010729.5,4.6,18.2,15.7,14.8,14.2,14.0,0.0789,SDSS,013412.80-010724.8,5.00,0.56,6.58,013412.84-010725.0,4.6,20.7,18.5,17.5,17.2,16.8,0.0784,SDSS,Grade A targets from Fu+2015.,013412.78-010729.6,013412.80-010724.8,01:34:12.78,-01:07:29.6,01:34:12.80,-01:07:24.8,23.553250,-1.124889,23.553333,-1.123556,J2000,Optical,SDSS,J2000,Optical,SDSS,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,Radio Imaging / Double Radio Sources,,Fu+2015,2015ApJ...799...72F,https://doi.org/10.1088/0004-637X/799/1/72
6,013638.30-002403.2,0.36,0.06,0.37,013638.29-002403.5,4.4,NaN,NaN,NaN,23.1,23.3,NaN,NaN,013638.53-002401.8,0.36,0.06,0.39,013638.56-002401.8,4.4,24.6,25.3,23.7,22.5,21.4,NaN,NaN,Grade A targets from Fu+2015.,013638.30-002403.2,013638.53-002401.8,01:36:38.30,-00:24:03.2,01:36:38.53,-00

In [41]:
#fu2015t3

secondary_objs = fu2015t3.iloc[1::2]#.reset_index(drop=True)
# Get the indices of rows in every_other_df
secondary_objs_indices = secondary_objs.index
# Drop rows from original_df
fu2015t3 = fu2015t3.drop(secondary_objs_indices).reset_index(drop=True)
secondary_objs = secondary_objs.reset_index(drop=True)
# Renaming columns for the secondary objects
secondary_objs.rename(columns={'RName':'RName2','Q':'Q2','z':'z2','Inst':'Inst2','aSep':'aSep2','pSep':'pSep2',\
                         'DelV':'DelV2','logP1.4':'logP1.4_2','l_logLHa':'l_logLHa2','logLHa':'logLHa2',\
                         'EWHa':'EWHa2','e_EWHa':'e_EWHa2','[NII]/Ha':'[NII]/Ha2','e_[NII]/Ha':'e_[NII]/Ha2',\
                         '[OIII]/Hb':'[OIII]/Hb2','e_[OIII]/Hb':'e_[OIII]/Hb2','Rcl':'Rcl2','BPTcl':'BPTcl2'}, \
                inplace=True)
fu2015t3.rename(columns={'RName':'RName1','Q':'Q1','z':'z1','Inst':'Inst1','aSep':'aSep1','pSep':'pSep1',\
                         'DelV':'DelV1','logP1.4':'logP1.4_1','l_logLHa':'l_logLHa1','logLHa':'logLHa1',\
                         'EWHa':'EWHa1','e_EWHa':'e_EWHa1','[NII]/Ha':'[NII]/Ha1','e_[NII]/Ha':'e_[NII]/Ha1',\
                         '[OIII]/Hb':'[OIII]/Hb1','e_[OIII]/Hb':'e_[OIII]/Hb1','Rcl':'Rcl1','BPTcl':'BPTcl1'}, \
                inplace=True)

fu2015t3 = pd.concat([fu2015t3,secondary_objs], axis=1).reset_index(drop=True)


In [42]:
fu2015t3

,RName1,Q1,z1,Inst1,aSep1,pSep1,DelV1,logP1.4_1,l_logLHa1,logLHa1,EWHa1,e_EWHa1,[NII]/Ha1,e_[NII]/Ha1,[OIII]/Hb1,e_[OIII]/Hb1,Rcl1,BPTcl1,RName2,Q2,z2,Inst2,aSep2,pSep2,DelV2,logP1.4_2,l_logLHa2,logLHa2,EWHa2,e_EWHa2,[NII]/Ha2,e_[NII]/Ha2,[OIII]/Hb2,e_[OIII]/Hb2,Rcl2,BPTcl2
0,005114.10+002049.5,A,0.1126,SDSS,3.4,7.1,58.7,23.0,NaN,42.4,51.0,0.3,0.6,0.1,0.3,0.1,Normal,Comp,005113.93+002047.0,A,0.1124,SDSS,3.4,7.1,58.7,22.3,NaN,41.8,23.6,0.4,0.7,0.1,1.0,0.2,Normal,Comp
1,013412.80-010724.8,A,0.0784,SDSS,4.6,6.8,136.9,23.0,<,39.4,-3.0,0.9,NaN,NaN,NaN,NaN,Excess,NaN,013412.78-010729.6,A,0.0789,SDSS,4.6,6.8,136.9,23.1,NaN,40.9,0.0,0.1,1.7,0.4,1.1,0.7,Excess,AGN2
2,220634.96+000327.6,A,0.0466,SDSS,4.6,4.2,124.5,22.0,NaN,41.2,8.7,0.2,0.8,0.1,0.7,0.1,Normal,Comp,220635.08+000323.3,A,0.0461,SDSS,4.6,4.1,124.5,22.1,NaN,41.6,14.8,0.2,1.5,0.1,1.3,0.1,Normal,AGN2
3,222051.66+005815.8,B,0.3178,LRIS,2.6,11.9,63.5,25.0,<,41.8,0.0,0.4,NaN,NaN,NaN,NaN,Excess,NaN,222051.44+005815.0,B,0.3181,SDSS,2.6,11.9,63.5,24.7,<,41.7,-2.0,1.2,NaN,NaN,NaN,NaN,Excess,NaN
4,223222.60+001224.8,A,0.2215,LRIS,3.2,11.6,122.8,23.2,NaN,41.5,35.6,0.3,0.5,0.1,1.3,0.2,Excess,Comp,223222.43+001225.8,A,0.2210,LRIS,3.2,11.6,122.8,22.9,NaN,41.2,28.3,0.3,0.6,0.1,0.6,0.1,Excess,Comp
5,230010.19-000531.8,A,0.1797,SDSS,2.5,7.7,20.9,23.2,<,40.5,-0.1,0.2,NaN,NaN,NaN,NaN,Excess,NaN,230010.31-000535.0,A,0.1798,SDSS,2.5,7.7,20.9,22.9,<,40.6,-1.5,0.3,NaN,NaN,NaN,NaN,Excess,NaN


In [43]:
fu2015t3['Name1'] = fu2015t3['RName1']
fu2015t3['Name2'] = fu2015t3['RName2']
        

fu2015t3['RA_test'] = fu2015t3['Name1'].str.slice(start=0, stop=9) # Stripping the DEC parts 
fu2015t3['Dec_test'] = fu2015t3['Name1'].str.slice(start=9, stop=19) # Stripping the RA parts
fu2015t3['RA'] = fu2015t3['RA_test'].str.slice(start=0, stop=2)+":"+fu2015t3['RA_test'].str.slice(start=2, stop=4)+":"+fu2015t3['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
fu2015t3['Dec'] = fu2015t3['Dec_test'].str.slice(start=0, stop=3)+":"+fu2015t3['Dec_test'].str.slice(start=3, stop=5)+":"+fu2015t3['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
fu2015t3.drop(columns=['RA_test','Dec_test'], inplace=True)


# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(fu2015t3,fu2015t3['Name1'])
fu2015t3.rename(columns={'RA': 'RA1','Dec':'Dec1'}, inplace=True)

fu2015t3['RA_test'] = fu2015t3['Name2'].str.slice(start=0, stop=9) # Stripping the DEC parts 
fu2015t3['Dec_test'] = fu2015t3['Name2'].str.slice(start=9, stop=19) # Stripping the RA parts
fu2015t3['RA'] = fu2015t3['RA_test'].str.slice(start=0, stop=2)+":"+fu2015t3['RA_test'].str.slice(start=2, stop=4)+":"+fu2015t3['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
fu2015t3['Dec'] = fu2015t3['Dec_test'].str.slice(start=0, stop=3)+":"+fu2015t3['Dec_test'].str.slice(start=3, stop=5)+":"+fu2015t3['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
fu2015t3.drop(columns=['RA_test','Dec_test'], inplace=True)

#name_to_coords(fu2015t3,fu2015t3['Name2'])
fu2015t3.rename(columns={'RA': 'RA2','Dec':'Dec2'}, inplace=True)

# Converting the coordinates
coordconvert = SkyCoord(ra = fu2015t3['RA1'], dec = fu2015t3['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
fu2015t3['RA1_deg'] = coordconvert.ra.degree
fu2015t3['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#fu2015['RA2'] = fu2015['RA']
#fu2015['Dec2'] = fu2015['Dec']

coordconvert = SkyCoord(ra = fu2015t3['RA2'], dec = fu2015t3['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
fu2015t3['RA2_deg'] = coordconvert.ra.degree
fu2015t3['Dec2_deg'] = coordconvert.dec.degree


#fu2015t3

In [44]:
#fu2015

In [45]:
# Here we're going to update the table 1 and 2 listings  using the table 3 listings

# Adding the DOI, author, and bibcode info to all of the Liu+2010 rows here in the matches table...
#for index, row in fu2015.iterrows():
#    if row['Name'] in fu2015t3['']:
#        tmatches.at[index, 'Paper(s)'] = 'Wang+2009 ; ' + tmatches.at[index, 'Paper(s)']
#        tmatches.at[index, 'BibCode(s)'] = '2009ApJ...705L..76W ; ' + tmatches.at[index, 'BibCode(s)']
#        tmatches.at[index, 'DOI(s)'] = 'https://doi.org/10.1088/0004-637X/705/1/L76 ; ' + tmatches.at[index, 'DOI(s)']

# Here we're going to match together the xliu2011 table and the Fu+2018 table and see if there are any matches

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(fu2015,fu2015t3,2)

print(len(tmatches_1),len(tmatches_2))
# same objects picked up in tmatches_1 and 2, except tmatches_1 pickes up a third object

fu2015['dV'] = -99

for i, j in zip(idx1_2, idx2_2):
    #print("i:", i, "j:", j)
    #print("Sep:", mcgurk2015t4.at[j, 'NIRC2sep(as)'])
    #print("dV:", mcgurk2015t4.at[j, 'dV[OIII]'])
    fu2015.at[i, 'dV'] = fu2015t3.at[j, 'DelV1']
    #fu2015.at[i, 'Notes'] += ' '

#tmatches_1



4 8


In [46]:
#tmatches_2

In [47]:
#fu2015

In [48]:
# Now loading in the information from Fu+2015b:

# The following two targets were also identified by Liu+2011 so we need to go back and mark those/remove them from these tables
# '005114.10+002049.5','220634.96+000327.6'

fu2015bobjs = ['005114.10+002049.5','220634.96+000327.6','223222.60+001224.8','230010.19-000531.8']

# fu2015b refers to obj two: 005114.11+002049.4 from the 2015b appear, but they used 005114.10+002049.5 in 2015a paper
# second object name matches object 1 from 2015
# second object name matches object 2 from 2015 
# fourth object matches object 1 from 2015

# Adding the DOI, author, and bibcode info 
for index, row in fu2015.iterrows():
    if (row['Name1'] in fu2015bobjs) or (row['Name2'] in fu2015bobjs):
        fu2015.at[index, 'Paper(s)'] += ' ; Fu+2015b'
        fu2015.at[index, 'BibCode(s)'] += ' ; 2015ApJ...815L...6F' 
        fu2015.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/2041-8205/815/1/L6'
        fu2015.at[index, 'System Type'] = 'Dual AGN'
        fu2015.at[index, 'Notes'] = 'Two compact steep-spectrum sources identified. '
        fu2015.at[index, 'Confirmation Method'] = 'Radio Imaging'

# Fu+ has HERG/LERG classifications and masses, so go back and add that in if we want it.        
        
fu2015bobjsbad = ['013412.80-010724.8','222051.66+005815.8']
# Adding the DOI, author, and bibcode info 
for index, row in fu2015.iterrows():
    if (row['Name1'] in fu2015bobjsbad) or (row['Name2'] in fu2015bobjsbad):
        fu2015.at[index, 'Paper(s)'] += ' ; Fu+2015b'
        fu2015.at[index, 'BibCode(s)'] += ' ; 2015ApJ...815L...6F' 
        fu2015.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1088/2041-8205/815/1/L6'
        fu2015.at[index, 'System Type'] = 'Single AGNs'
        fu2015.at[index, 'Notes'] = 'Faranoff-Riley II radio galaxies located within a galaxy merger.'
        fu2015.at[index, 'Confirmation Method'] = 'Radio Imaging'
# We'll want to throw these out...



# Now loading in the information from Gross+2019:
gross2019objs = ['005114.10+002049.5','220634.96+000327.6','223222.60+001224.8','230010.19-000531.8']

# Gross used 005114.10+002049.5, but should have been 005114.10+002049.5

# Adding the DOI, author, and bibcode info
for index, row in fu2015.iterrows():
    if (row['Name1'] in gross2019objs) or (row['Name2'] in gross2019objs):
        fu2015.at[index, 'Paper(s)'] += ' ; Gross+2019'
        fu2015.at[index, 'BibCode(s)'] += ' ; 2019ApJ...883...50G' 
        fu2015.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/ab3795'
        #fu2015.at[index, 'System Type']='Dual AGN'
        #fu2015.at[index, 'Notes']=' '
        #fu2015.at[index, 'Confirmation Method'] = ' '


# Need to come back here and add some details from Gross+2019, but it is not critical right now (31 July 2023)


# verified that this matching process is working properly


In [49]:
#fu2015

In [50]:
#xliu2011

In [51]:
# Now we're going to match up the overarching xliu2011+fu2018 table with fu2015. There should be two matches

tunique_1, tmatches_1, idx1, idx2, tunique_2, tmatches_2, idx1_2, idx2_2 = match_tables_fib(xliu2011,fu2015,3)

print(len(tmatches_1),len(tmatches_2))
# same objects picked up in tmatches_1 and 2, except tmatches_1 pickes up a third object

#fu2015['dV'] = -99

# We are going to defer to Liu+ for naming, coordinates, separation. Redshifts are identical.
# We need to add author, bibcodes, dois, notes, and selection methods from Fu+

for i, j in zip(idx1, idx2):
    xliu2011.at[i, 'Paper(s)'] += ' ; ' + fu2015.at[j, 'Paper(s)']
    xliu2011.at[i, 'BibCode(s)'] += ' ; ' + fu2015.at[j, 'BibCode(s)']
    xliu2011.at[i, 'DOI(s)'] += ' ; ' + fu2015.at[j, 'DOI(s)']
    xliu2011.at[i, 'Selection Method'] += ' / ' + fu2015.at[j, 'Selection Method']
    xliu2011.at[i, 'Confirmation Method'] += ' / ' + fu2015.at[j, 'Confirmation Method']
    xliu2011.at[i, 'Notes'] += ' ' + fu2015.at[j, 'Notes']
    xliu2011.at[i, 'System Type'] += ' / ' + fu2015.at[j, 'System Type']

fu2015.drop(idx2).reset_index(drop=True, inplace=True)

# verified that this works, but there is an issue with spacing and semi colons. We need to go back in \
# and add semi colons and spaces so that the formatting looks fine.


4 0


In [52]:
# And finally concatenating the overarching table and Fu+2015

xliu2011 = pd.concat([xliu2011,fu2015])
xliu2011.reset_index(drop=True, inplace=True)


In [53]:
# now formatting the table before saving
xliu2011.drop(labels=['level_0','index','RName1','S1.4pk1','e_S1.4pk1','S1.4int1','OName1','umag1','gmag1','imag1','zmag1',\
                     'Inst1','RName2','S1.4pk2','e_S1.4pk2','S1.4int2','OName2','Sep2','umag2','gmag2',\
                     'imag2','zmag2','Inst2','Table_flag','Plate','IFU','Sample','Sep_as','rp','delta_v',\
                     'R.A.1','Decl.1','Redshift1','stellar_disp1','log(M)1','log(L[OIII])1','Class1',\
                     'R.A.2','Decl.2','Redshift2','stellar_disp2','log(M)2','log(L[OIII])2','Class2'], inplace=True, axis=1)


In [54]:
xliu2011

,SDSS1,z1,dV,rmag1,RA1,Dec1,SDSS2,z2,rmag2,RA2,Dec2,System Type,Selection Method,Confirmation Method,Name1,Name2,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Equinox1,Coordinate_waveband1,Coordinate_Source1,Equinox2,Coordinate_waveband2,Coordinate_Source2,Sep,Sep(kpc),dV_rwp,z1_type,z2_type,Paper(s),BibCode(s),DOI(s),Notes,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2
0,J000249.07+004504.8,0.08680,63.000000,16.08,00:02:49.07,+00:45:04.8,J000249.44+004506.7,0.08650,16.43,00:02:49.44,+00:45:06.7,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000249.07+004504.8,J000249.44+004506.7,0.704458,0.751333,0.706000,0.751861,J2000,Optical,SDSS,J2000,Optical,SDSS,5.865765,9.534117,82.547277,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
1,J000257.21+000750.5,0.09010,109.000000,16.15,00:02:57.21,+00:07:50.5,J000258.59+000831.0,0.08970,16.67,00:02:58.59,+00:08:31.0,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000257.21+000750.5,J000258.59+000831.0,0.738375,0.130694,0.744125,0.141944,J2000,Optical,SDSS,J2000,Optical,SDSS,45.483377,76.446410,109.734834,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
2,J000323.74+010547.3,0.09930,22.000000,17.98,00:03:23.74,+01:05:47.3,J000323.74+010559.5,0.09940,17.42,00:03:23.74,+01:05:59.5,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000323.74+010547.3,J000323.74+010559.5,0.848917,1.096472,0.848917,1.099861,J2000,Optical,SDSS,J2000,Optical,SDSS,12.200000,22.360857,-27.197890,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
3,J000425.78-095854.4,0.11220,53.000000,16.41,00:04:25.78,-09:58:54.4,J000426.66-095834.4,0.11200,16.57,00:04:26.66,-09:58:34.4,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000425.78-095854.4,J000426.66-095834.4,1.107417,-9.981778,1.111083,-9.976222,J2000,Optical,SDSS,J2000,Optical,SDSS,23.853885,48.677088,53.772142,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
4,J000431.92-011411.7,0.08870,45.000000,17.22,00:04:31.92,-01:14:11.7,J000433.25-011334.4,0.08890,18.47,00:04:33.25,-01:13:34.4,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000431.92-011411.7,J000433.25-011334.4,1.133000,-1.236583,1.138542,-1.226222,J2000,Optical,SDSS,J2000,Optical,SDSS,42.297857,70.100953,-54.922850,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
5,J000621.33-100412.6,0.10620,165.000000,17.46,00:06:21.33,-10:04:12.6,J000623.74-100345.0,0.10560,17.34,00:06:23.74,-10:03:45.0,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000621.33-100412.6,J000623.74-100345.0,1.588875,-10.070167,1.598917,-10.062500,J2000,Optical,SDSS,J2000,Optical,SDSS,45.040625,87.594159,162.220804,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
6,J000700.56+005410.5,0.09250,250.000000,15.44,00:07:00.56,+00:54:10.5,J000704.18+005425.0,0.09350,16.96,00:07:04.18,+00:54:25.0,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000700.56+005410.5,J000704.18+005425.0,1.752333,0.902917,1.767417,0.906944,J2000,Optical,SDSS,J2000,Optical,SDSS,56.196126,96.699700,-273.558955,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737/2/101,,NaN,NaN,NaN,NaN,NaN,NaN
7,J000948.47+135819.8,0.11790,281.000000,17.39,00:09:48.47,+13:58:19.8,J000948.49+135811.9,0.11690,17.92,00:09:48.49,+13:58:11.9,Dual AGN Candidate,Fiber Spectroscopy / Optical Spectroscopy / Op...,,J000948.47+135819.8,J000948.49+135811.9,2.451958,13.972167,2.452042,13.969972,J2000,Optical,SDSS,J2000,Optical,SDSS,7.905362,16.841984,267.585413,spec,spec,Liu+2011b,2011ApJ...737..101L,https://doi.org/10.1088/0004-637X/737

In [55]:
# now saving the duals table...
xliu2011.to_csv('Opticalspec_radiosel_DR1.csv', sep=',', index=False)

# there are a few hiccups when info is added from Fu to Liu, but for now this is working
# JUST COME BACK AND FIX

